In [1]:
import urllib3
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

### https://github.com/CrowdTangle/API/wiki/Posts
### https://help.crowdtangle.com/en/articles/1189612-crowdtangle-api

In [2]:
token = ""

In [3]:
dataset=pd.DataFrame()

#### Key: Made a list of keywords for which I need the reddit posts

In [7]:
#key=["covid", "coronavirus","covid19","covid-19","hospital","mask","death","infection","quarantine","virus","symptom","flu","smell","cough","fever","circuitbreaker","wuhan virus","chinese virus"]
key=["%23stayathome", "%23sgunited","%23circuitbreakersg","%23stayhomesg","%23sgcircuitbreaker","%23circuitbreaker","%23sgunited", "%23stayhomeforsg", "%23movementcontrolorder", "%23safedistancing","%23igsingapore","%23sgtogether","%23socialdistancing","%23lockdown"]
userkey=["302218","710418"]

In [ ]:
for i in range(1,len(userkey)):
    print(i, userkey[i])
#    for keywords: nexturl = 'https://api.crowdtangle.com/posts?token='+token+'&sortBy=total_interactions&endDate=2020-04-18T00:00:00&startDate=2020-01-01T00:00:00&searchTerm='+key[i]+'&count=100'
# for posts by users:
    nexturl = 'https://api.crowdtangle.com/posts/search?token='+token+'&inAccountIds='+ userkey[i]+'&sortBy=total_interactions&startDate=2020-02-01T00:00:00&endDate=2020-06-02T00:00:00&searchTerm=i&count=100'
    while(nexturl.strip()):
        time.sleep(20)
        print(nexturl)
        response = requests.get(nexturl)    
        dict = response.json()
        
        if 'result' in dict.keys():
            df = pd.DataFrame(dict['result']['posts'])
            df["Label"]='all'
            dataset = pd.concat([dataset, df], ignore_index=True)
            if 'nextPage' in dict['result']['pagination'].keys():
                nexturl = dict['result']['pagination']['nextPage']
            else:
                nexturl=''
        else:
            print(dict)
            nexturl=''
    

#### Number of posts fetched


In [9]:
dataset.shape

(14, 18)

#### Count of number of posts for each keyword

In [10]:
dataset.Label.value_counts()

all    14
Name: Label, dtype: int64

#### Saving csv

In [9]:
dataset.to_csv('sing_allaccounts_jun18.csv', index=False) 

#### Now create a list of ids of all the posts fetched. This will be used to get the comments of the posts fetched above

In [48]:
ids = list(dataset['id'])

In [52]:
len(ids)

10880

#### Now traverse through the "ids" list to fetch the comment data one by one.
#### In this case also, all the commented data is for each post is appended together to form a single comment CSV file i.e. "dataset2"
Code below


In [74]:
dataset2=pd.DataFrame()

In [ ]:
for i in range(0,10880):
    print(i)
    url = 'http://api.pushshift.io/reddit/comment/search/?link_id='+str(ids[i])+'&size=50&sort_type=created_utc&sort=asc&filter=author,body,created_utc,score,subreddit,no_follow&score=>-100'
    response = requests.get(url)    
    dict = response.json()
    df = pd.DataFrame(dict['data'])
    df["id"]=str(ids[i])
    dataset2 = pd.concat([dataset2, df], ignore_index=True)

#### Saving comment data as CSV

In [ ]:
dataset2.to_csv('comment_dataset.csv', index=False) 